# Data used to validate pyKNEEr
Content under Creative Commons Attribution license CC-BY-NC 4.0   
Code under GNU-GPL v3 License  
© 2019 Serena Bonaretti
---

## Image characteristics
Data characteristics are extracted from the files *_orig.txt* created in the preprocessing step

In [1]:
import numpy  as np
import pandas as pd
import os

### Functions to convert *_orig.txt* files to a pandas dataframe

In [2]:
def dicom_tags(): 
    '''
    list of tags of interest 
    ''' 
    # list with the tags of interest
    tags_of_interest = []
    
    # image characteristics
    tags_of_interest.append(["0018|0050", "SliceThickness"])
    tags_of_interest.append(["0018|0088", "SpacingBetweenSlices"])
    tags_of_interest.append(["0028|0030", "PixelSpacing"])
    tags_of_interest.append(["0028|0010", "Rows"])
    tags_of_interest.append(["0028|0011", "Columns"])
    
    # acquisition characteristics
    tags_of_interest.append(["0018|0080", "RepetitionTime"])
    tags_of_interest.append(["0018|0081", "EchoTime"])
    tags_of_interest.append(["0018|0086", "EchoNumbers"])
    tags_of_interest.append(["0018|1314", "FlipAngle"])
    tags_of_interest.append(["0018|1030", "ProtocolName"])
    tags_of_interest.append(["0008|103e", "SeriesDescription"])
    tags_of_interest.append(["0018|0023", "MRAcquisitionType"])
    tags_of_interest.append(["0018|0084", "ImagingFrequency"])
    
    # hardware characteristics
    tags_of_interest.append(["0008|0060", "Modality"]) 
    tags_of_interest.append(["0008|0070", "Manufacturer"])
    tags_of_interest.append(["0008|0080", "InstitutionName"])
    tags_of_interest.append(["0008|1090", "ManufacturerModelName"])
    tags_of_interest.append(["0018|1020", "SoftwareVersions"])
    tags_of_interest.append(["0018|1250", "ReceiveCoilName"])
    tags_of_interest.append(["0018|0087", "MagneticFieldStrength"])
    
    # subject characteristics
    tags_of_interest.append(["0010|0020", "PatientID"])
    tags_of_interest.append(["0010|0030", "PatientBirthDate"])
    tags_of_interest.append(["0010|0040", "PatientSex"])
    tags_of_interest.append(["0010|1010", "PatientAge"])
    tags_of_interest.append(["0010|1030", "PatientWeight"])
    
    return tags_of_interest

In [3]:
def from_file_to_list(folder, file_name):
    '''
    read and parse *_orig.txt, and find the values corresponding to "tags_of_interest" 
    '''
    
    # get the list of tags of interest
    tags_of_interest = dicom_tags()
    # extract the numerical tags (first column of tags_of_interest)
    num_tags         = [row[0] for row in tags_of_interest]

    # get the dicom tags in the current file
    
    tags   = [] # list of dicom ID found in num_tags
    values = [] # content of *_orig.txt (each element is a file row) 
    
    # get all values and tags
    for line in open(folder + file_name):
        # get the whole row
        row = line.rstrip("\n")
        # get the dicom tag
        tags.append(row[0:9])
        # get the value corresponding to the tag
        values.append(row[10:len(row)])
        
    # select the values corresponding to the tags of interest
    values_of_interest = [] # values corresponding to tags of interest
    # browse the tags of interest (they are less than the image tags)
    for current_tag in num_tags:
        # if a tag of interest is present in the current dicom header, get the value
        if current_tag in tags:
            # get the index
            index = tags.index(current_tag)
            # get the value
            values_of_interest.append(values[index])
        # otherwise write "NA"
        else:
            values_of_interest.append("NA")

    return values_of_interest


In [4]:
def create_data_frame(folder):
    
    # get the list of tags of interest
    tags_of_interest = dicom_tags()
    # extract the string tags (second column of tags_of_interest)
    tags_string      = [row[1] for row in tags_of_interest]
    # add file name of *_orig.txt
    tags_string.append("fileName")
    
    # get the values corresponding to the tags of interest

    # find all *_orig.txt files in the folder
    file_names_list = []
    for file_name in os.listdir(folder):
        if file_name.endswith("_orig.txt") and file_name[0] != ".": # !=  "." is to exclude hidden files
            file_names_list.append(file_name)

    # get the values corresponding to tags
    all_data = []
    for file_name in file_names_list:
        vector = from_file_to_list(folder, file_name)
        # add the *_orig.txt filename
        file_name_root, file_name_ext = os.path.splitext(file_name)
        vector.append(file_name_root)
        # assign vector to bigger matrix
        all_data.append(vector)
    
    # put values in pandas dataframe
    df = pd.DataFrame.from_records(all_data)
    # assign column labels
    df.columns = tags_string
    # start index from 1
    df.index = np.arange(1,len(df)+1) # start counting rows from 1
    # move file name to first column
    col = df['fileName']
    df.drop(labels=['fileName'], axis=1, inplace=True)
    df.insert(0,'fileName',col)
    # show all rows and columns
    dataDimension = df.shape # get number of rows
    pd.set_option("display.max_rows",dataDimension[0])
    #pd.set_option("display.max_columns",dataDimension[1])
    
    return df

---

## Load data
The following folder paths point to a harddrive. The folders contain the same images as the folders in Zenodo

In [5]:
# OAI1
folder_OAI1  = "/Volumes/SereHD/work/data_pyKNEEr/OAI1/preprocessed/"
df_OAI1      = create_data_frame(folder_OAI1)
df_OAI1.name = "OAI1" # name dataframe for print in queries
#display(df_OAI1)

In [6]:
# OAI2
folder_OAI2  = "/Volumes/SereHD/work/data_pyKNEEr/OAI2/preprocessed/"
df_OAI2      = create_data_frame(folder_OAI2)
df_OAI2.name = "OAI2" # name dataframe for print in queries
#display(df_OAI2)

In [7]:
# in-house
folder_inHouse  = "/Volumes/SereHD/work/data_pyKNEEr/inHouse/preprocessed/"
df_inHouse      = create_data_frame(folder_inHouse)
df_inHouse.name = "inHouse" # name dataframe for print in queries
#display(df_inHouse)

In [8]:
# dataframes in list for for loops in queries
dfs = [df_OAI1,df_OAI2,df_inHouse]

--- 

## Query data

### Q: How many *dicom stacks* per dataset?

In [9]:
n_OAI1    = df_OAI1.describe()["fileName"]["count"]
n_OAI2    = df_OAI2.describe()["fileName"]["count"]
n_inHouse = df_inHouse.describe()["fileName"]["count"]
print("Number of dicom stacks: ")
print("  OAI1   : " + str(n_OAI1))
print("  OAI2   : " + str(n_OAI2))
print("  inHouse: " + str(n_inHouse))
print("  Total  : " + str(n_OAI1 + n_OAI2 + n_inHouse))

Number of dicom stacks: 
  OAI1   : 152
  OAI2   : 176
  inHouse: 24
  Total  : 352


### Q: How many *subjects* per dataset?

In [10]:
q      = df_OAI1.groupby('PatientID').count()
n_OAI1 = q.shape[0] # get number of rows
q      = df_OAI2.groupby('PatientID').count()
n_OAI2 = q.shape[0]
# q      = df_inHouse.groupby('PatientID').count()  # The inHouse images were anonymized deleting also the PatientID,
# n_inHouse = q.shape[0]                            # so they cannot be included in this count
print("Number of subjects: ")
print("  OAI1   : " + str(n_OAI1))
print("  OAI2   : " + str(n_OAI2))
print("  Total  : " + str(n_OAI1 + n_OAI2))

Number of subjects: 
  OAI1   : 19
  OAI2   : 88
  Total  : 107


### Q: What is the in-plane resolution per protocol?

In [11]:
for df_index in range (0,len(dfs)):
    # query the dataframe
    q = dfs[df_index].groupby('PixelSpacing')['SeriesDescription'].value_counts().reset_index(name='counts')
    # start counting rows from 1
    q.index = np.arange(1,len(q)+1) 
    # display dataframe name
    display(dfs[df_index].name)
    # display query output
    display(q)

'OAI1'

,PixelSpacing,SeriesDescription,counts
1,0.3125\0.3125,SAG_T2_MAP_RIGHT,126
2,0.36458333333333\0.36458333333333,SAG_3D_DESS_RIGHT,14
3,0.36458334326744\0.36458334326744,SAG_3D_DESS_RIGHT,4
4,0.42708333333333\0.42708333333333,SAG_3D_DESS_RIGHT,1
5,0.4296875\0.4296875,SAG_T2_MAP_RIGHT,7


'OAI2'

,PixelSpacing,SeriesDescription,counts
1,0.36458333\0.36458333,SAG_3D_DESS_RIGHT,90
2,0.36458333\0.36458333,SAG_3D_DESS_LEFT,86


'inHouse'

,PixelSpacing,SeriesDescription,counts
1,0.3125\0.3125,cubequant T1rho,16
2,0.3125\0.3125,Sag DESS 3132 30 HiRes thk1p5 ARCx2,8


### Q: What is the slice thickness per protocol?

In [12]:
for df_index in range (0,len(dfs)):
    # query the dataframe
    q = dfs[df_index].groupby('SliceThickness')['SeriesDescription'].value_counts().reset_index(name='counts')
    # start counting rows from 1
    q.index = np.arange(1,len(q)+1) 
    # display dataframe name
    display(dfs[df_index].name)
    # display query output
    display(q)

'OAI1'

,SliceThickness,SeriesDescription,counts
1,0.69999998807907,SAG_3D_DESS_RIGHT,18
2,0.75,SAG_3D_DESS_RIGHT,1
3,3,SAG_T2_MAP_RIGHT,119
4,3.0,SAG_T2_MAP_RIGHT,7
5,3.5,SAG_T2_MAP_RIGHT,7


'OAI2'

,SliceThickness,SeriesDescription,counts
1,0.69999999,SAG_3D_DESS_RIGHT,90
2,0.69999999,SAG_3D_DESS_LEFT,86


'inHouse'

,SliceThickness,SeriesDescription,counts
1,1.5,Sag DESS 3132 30 HiRes thk1p5 ARCx2,8
2,3,cubequant T1rho,16


### Q: What is the echo time? 

In [13]:
for df_index in range (0,len(dfs)):
    # query the dataframe
    q = dfs[df_index].groupby('EchoTime')['SeriesDescription'].value_counts().reset_index(name='counts')
    # start counting rows from 1
    q.index = np.arange(1,len(q)+1) 
    # display dataframe name
    display(dfs[df_index].name)
    # display query output
    display(q)

'OAI1'

,EchoTime,SeriesDescription,counts
1,10,SAG_T2_MAP_RIGHT,21
2,10.0,SAG_T2_MAP_RIGHT,2
3,20,SAG_T2_MAP_RIGHT,18
4,20.0,SAG_T2_MAP_RIGHT,1
5,30,SAG_T2_MAP_RIGHT,18
6,30.0,SAG_T2_MAP_RIGHT,1
7,4.71,SAG_3D_DESS_RIGHT,15
8,4.72,SAG_3D_DESS_RIGHT,4
9,40,SAG_T2_MAP_RIGHT,18
10,40.0,SAG_T2_MAP_RIGHT,1


'OAI2'

,EchoTime,SeriesDescription,counts
1,4.71,SAG_3D_DESS_RIGHT,90
2,4.71,SAG_3D_DESS_LEFT,86


'inHouse'

,EchoTime,SeriesDescription,counts
1,1,cubequant T1rho,4
2,10,cubequant T1rho,4
3,30,cubequant T1rho,4
4,42.52,Sag DESS 3132 30 HiRes thk1p5 ARCx2,4
5,60,cubequant T1rho,4
6,7.48,Sag DESS 3132 30 HiRes thk1p5 ARCx2,4


### Q: What is the repetition time? 

In [14]:
for df_index in range (0,len(dfs)):
    # query the dataframe
    q = dfs[df_index].groupby('RepetitionTime')['SeriesDescription'].value_counts().reset_index(name='counts')
    # start counting rows from 1
    q.index = np.arange(1,len(q)+1) 
    # display dataframe name
    display(dfs[df_index].name)
    # display query output
    display(q)

'OAI1'

,RepetitionTime,SeriesDescription,counts
1,16.32,SAG_3D_DESS_RIGHT,19
2,2700,SAG_T2_MAP_RIGHT,119
3,2700.0,SAG_T2_MAP_RIGHT,7
4,2900,SAG_T2_MAP_RIGHT,7


'OAI2'

,RepetitionTime,SeriesDescription,counts
1,16.32,SAG_3D_DESS_RIGHT,90
2,16.32,SAG_3D_DESS_LEFT,86


'inHouse'

,RepetitionTime,SeriesDescription,counts
1,1301,cubequant T1rho,4
2,1304,cubequant T1rho,8
3,1306,cubequant T1rho,4
4,25,Sag DESS 3132 30 HiRes thk1p5 ARCx2,8


### Q: What is the flip angle?

In [15]:
for df_index in range (0,len(dfs)):
    # query the dataframe
    q = dfs[df_index].groupby('FlipAngle')['SeriesDescription'].value_counts().reset_index(name='counts')
    # start counting rows from 1
    q.index = np.arange(1,len(q)+1) 
    # display dataframe name
    display(dfs[df_index].name)
    # display query output
    display(q)

'OAI1'

,FlipAngle,SeriesDescription,counts
1,180,SAG_T2_MAP_RIGHT,126
2,180.0,SAG_T2_MAP_RIGHT,7
3,25,SAG_3D_DESS_RIGHT,18
4,25.0,SAG_3D_DESS_RIGHT,1


'OAI2'

,FlipAngle,SeriesDescription,counts
1,25,SAG_3D_DESS_RIGHT,90
2,25,SAG_3D_DESS_LEFT,86


'inHouse'

,FlipAngle,SeriesDescription,counts
1,30,Sag DESS 3132 30 HiRes thk1p5 ARCx2,8
2,90,cubequant T1rho,16


### Dependencies

In [16]:
%load_ext watermark
%watermark -v -m -p numpy,pandas

CPython 3.7.1
IPython 6.5.0

numpy 1.15.1
pandas 0.23.4

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.7.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
